## 1. Get preprocessed data with a help of Dask

In [2]:
from ydata_synthetic.evaluate import evaluator
from ydata_synthetic.synthesizers.regular import wgangp
from ydata_synthetic.preprocessing.adult_dask import transformations
from ydata_synthetic.synthesizers.regular.wgangp import model

In [3]:
processed_data, preprocessor = transformations()

In [4]:
processed_data

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,workclass_0,workclass_1,workclass_2,workclass_3,workclass_4,...,relationship_4,relationship_5,race_0,race_1,race_2,race_3,race_4,sex_0,sex_1,target
0,0.025996,-1.061979,0.146932,-0.217127,-0.034087,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
1,0.828308,-1.007104,-0.144804,-0.217127,-2.213032,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
2,-0.046942,0.246034,-0.144804,-0.217127,-0.034087,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
3,1.047121,0.426663,-0.144804,-0.217127,-0.034087,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
4,-0.776316,1.408530,-0.144804,-0.217127,-0.034087,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.025996,0.243884,-0.144804,-0.217127,-0.356894,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
48838,1.849433,1.247492,-0.144804,-0.217127,-0.034087,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
48839,-0.046942,1.754865,-0.144804,-0.217127,0.772930,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
48840,0.390683,-1.001612,0.587220,-0.217127,-0.034087,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1


## 2. Evaluate data

###  2.1 Evaluate real data vs real data

In [12]:
eval_class = evaluator.EvaluateData(processed_data, processed_data, 20000)
score_pca_real = eval_class.pca_correlation()
results, score_ml_real = eval_class.set_ml_estimation()

Top 5 PCA components:
   real_data_pca  fake_data_pca
0       1.522808       1.513951
1       1.055745       1.027526
2       1.020751       0.999268
3       0.944357       0.926584
4       0.906156       0.868855


In [13]:
print("ML evaluation score", score_ml_real)
print("PCA evaluation score", score_pca_real)

ML evaluation score 0.9148035116085675
PCA evaluation score 0.9772851075151203


In [14]:
results

,real_data_f1,fake_data_f1
index,,
LogisticRegression_real_test,0.851333,0.849467
LogisticRegression_fake_test,0.852867,0.853267
DecisionTreeClassifier_real_test,1.000000,0.866400
DecisionTreeClassifier_fake_test,0.868600,1.000000
RandomForestClassifier_real_test,0.993267,0.885867
RandomForestClassifier_fake_test,0.890533,0.993400


###  2.1 Evaluate real data vs fake data

In [15]:
# WGAN_GP training
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 200+1
learning_rate = 5e-4
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

gan_args = [batch_size, learning_rate, beta_1, beta_2, noise_dim, processed_data.shape[1], dim]
train_args = ['', epochs, log_step]

synthesizer = model.WGAN_GP(gan_args, n_critic=2)
synthesizer.train(processed_data, train_args)

#Generate data
synth_data = synthesizer.sample(10000)


  2%|███▍                                                                                                                                                                           | 4/201 [00:02<01:20,  2.44it/s]

Iteration: 0 | disc_loss: 38.936058044433594 | gen_loss: 0.010258849710226059
Iteration: 1 | disc_loss: 20.809507369995117 | gen_loss: 0.008445937186479568
Iteration: 2 | disc_loss: 15.737884521484375 | gen_loss: 0.005744484253227711
Iteration: 3 | disc_loss: 11.074752807617188 | gen_loss: 0.00559468287974596


  4%|██████▉                                                                                                                                                                        | 8/201 [00:02<00:34,  5.64it/s]

Iteration: 4 | disc_loss: 9.2372407913208 | gen_loss: 0.008237235248088837
Iteration: 5 | disc_loss: 6.975252628326416 | gen_loss: 0.007715933956205845
Iteration: 6 | disc_loss: 5.687011241912842 | gen_loss: 0.008811427280306816
Iteration: 7 | disc_loss: 4.450337886810303 | gen_loss: 0.02015542984008789
Iteration: 8 | disc_loss: 3.510256290435791 | gen_loss: 0.019585872069001198


  6%|██████████▍                                                                                                                                                                   | 12/201 [00:02<00:20,  9.10it/s]

...

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉      | 194/201 [00:11<00:00, 20.89it/s]

Iteration: 190 | disc_loss: -0.1276920884847641 | gen_loss: 0.0590265691280365
Iteration: 191 | disc_loss: -0.11511756479740143 | gen_loss: 0.045792028307914734
Iteration: 192 | disc_loss: -0.12603576481342316 | gen_loss: 0.05303044989705086
Iteration: 193 | disc_loss: -0.127083420753479 | gen_loss: 0.050557561218738556
Iteration: 194 | disc_loss: -0.10821297764778137 | gen_loss: 0.05444476008415222


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 200/201 [00:12<00:00, 22.34it/s]

Iteration: 195 | disc_loss: -0.11073752492666245 | gen_loss: 0.0547836497426033
Iteration: 196 | disc_loss: -0.13001839816570282 | gen_loss: 0.028757404536008835
Iteration: 197 | disc_loss: -0.12803515791893005 | gen_loss: 0.05500848591327667
Iteration: 198 | disc_loss: -0.12279795855283737 | gen_loss: 0.042554549872875214
Iteration: 199 | disc_loss: -0.12863697111606598 | gen_loss: 0.03555373102426529
Iteration: 200 | disc_loss: -0.13135895133018494 | gen_loss: 0.04878925159573555


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 301.19it/s]


In [16]:
# Rename target variable to make it uniformed
synth_data.iloc[:,-1] = synth_data.iloc[:,-1].round()
synth_data.rename(columns={synth_data.columns[-1]: "target"}, inplace = True)

In [17]:
eval_class = evaluator.EvaluateData(processed_data, synth_data, 10000)
score_pca = eval_class.pca_correlation()
results, score_ml = eval_class.set_ml_estimation()

Top 5 PCA components:
   real_data_pca  fake_data_pca
0       1.503980       0.139497
1       1.055128       0.102808
2       1.013307       0.082029
3       0.936112       0.047326
4       0.905934       0.032910


In [18]:
results

,real_data_f1,fake_data_f1
index,,
LogisticRegression_real_test,0.851333,0.672400
LogisticRegression_fake_test,0.980000,0.983733
DecisionTreeClassifier_real_test,0.999867,0.601067
DecisionTreeClassifier_fake_test,0.980000,1.000000
RandomForestClassifier_real_test,0.993467,0.394267
RandomForestClassifier_fake_test,0.980000,0.999467


In [21]:
print("ML evaluation score", score_ml)
print("PCA evaluation score", score_pca)

ML evaluation score 0.7906240932066106
PCA evaluation score 0.07160467472780285


The PCA score seeems to be that low because categorical variable are generated the same way as numerical.

## 3. Advice on WGAN-GP

1. Right now we sample initial fake data for the categorical variables the same way as for the numerical. This leads to some bizarre values in categorical columns. We should use something like Gumbel-Softmax distribution to sample from. This way we will get real one-hot encoded categorical outputs.
2. In data processing we should also add a target variable, not only features.
3. The architecture itself could be improved by adding a batch normalization layer.
4. Another idea is to use Banach Wasserstein GAN architecture, which was not really done to tabular data, but shows some improvements in picture generation.